In [ ]:
""" 25-03-10 지금 몇몇 라인에서 개행문자를 그대로 가져와서, CSV 파일이 깨지는 문제가 발생... 해당 문제를 해결하기 위한 파싱 코드
    CPG를 csv 파일로 변환하는 해당 코드에서 생각보다 많은 시간을 소요한다.
    
    1. 파일 입출력
    2. regex의 빈번한 사용
    3. 불필요한 조건식 등

"""
import json
import csv
import re
import os

# JSON 파일을 읽어오는 함수
def read_json_file(file_path):
    with open(file_path, 'r') as file:
        return json.load(file)

# Edges를 CSV 파일로 추출
def write_edges_to_csv(edges, output_file):
    Function_ids = {}
    with open(output_file, mode='w', newline='') as file:
        writer = csv.writer(file, delimiter="\t")
        
        # CSV 파일 헤더 작성 
        writer.writerow(["start", "startType", "end", "endType", "type", "var"])   
        
        # 각 edge 정보 추출 및 CSV 파일에 작성
        for edge in edges:
            start = edge["outV"]["@value"]
            startType = edge["outVLabel"]
            end = edge["inV"]["@value"]
            endType = edge["inVLabel"]
            type = edge["label"]
            
            if type == "CONTAINS" and edge["inVLabel"] == "CALL":
                Function_ids[end] = start
            
            if "EdgeProperty" in edge['properties']: 
                var = edge['properties']['EdgeProperty']['@value']
            else:
                var = ""
            writer.writerow([start, startType, end, endType, type, var])    
    return Function_ids

# Vertices(노드)를 CSV 파일로 추출
def write_vertices_to_csv(vertices, Function_ids, output_file):
    with open(output_file, mode='w', newline='') as file:
        writer = csv.writer(file, delimiter="\t")
        writer.writerow(["key", "type", "code", "name", "location", "locationEnd", "functionId", "argIndex", "isExternal"])
        
        for vertex in vertices:
            key = vertex["id"]["@value"]
            type = vertex["label"]

            # CODE 프로퍼티 처리 (개행 제거 + 연속 공백 압축)
            if "CODE" in vertex["properties"] and vertex["properties"]["CODE"]["@value"]["@value"] not in ["<global>", "<empty>"]:
                code_list = vertex["properties"]["CODE"]["@value"]["@value"]
                code_string = " ".join([str(item).replace("\n", " ").replace("\r", " ") for item in code_list if item])
                code_string = re.sub(r'\s+', ' ', code_string)  # 연속된 공백을 하나로 압축
                code = code_string
            else:
                code = ""

            if key in Function_ids.values():
                match = re.search(r'\b\w+\s+(\w+)\s*\(.*\)', code)
                if match:
                    code = match.group(1)
                else:
                    code = ""

            elif type == "BLOCK":
                code = ""

            elif type == "TYPE_DECL":
                code_list = vertex["properties"]["CODE"]["@value"]["@value"]
                code_string = " ".join([str(item).replace("\n", " ") for item in code_list if item])
                code_string = re.sub(r'([()*,\.])', r' \1 ', code_string)
                code = code_string

            else:
                if "CODE" in vertex["properties"]:
                    code_list = vertex["properties"]["CODE"]["@value"]["@value"]
                    code_string = " ".join([str(item).replace("\n", " ").replace("\r", " ") for item in code_list if item])
                    code_string = re.sub(r'\s+', ' ', code_string)  # 연속된 공백을 하나로 압축
                    code_string = re.sub(r'([()*,\.])', r' \1 ', code_string)  # 괄호, 쉼표, 별표, 마침표 앞뒤에 공백 추가
                    code = code_string
                else:
                    code = ""

            if "NAME" in vertex["properties"]:
                name = vertex["properties"]["NAME"]["@value"]["@value"][0]
            else:
                name = ""

            if "LINE_NUMBER" in vertex["properties"]:
                location = vertex["properties"]["LINE_NUMBER"]["@value"]["@value"][0]["@value"]
            else:
                location = ""

            if "LINE_NUMBER_END" in vertex["properties"]:
                locationEnd = vertex["properties"]["LINE_NUMBER_END"]["@value"]["@value"][0]["@value"]
            else:
                locationEnd = ""

            functionId = Function_ids.get(key, "")

            if "ARGUMENT_INDEX" in vertex["properties"]:
                argIndex = vertex["properties"]["ARGUMENT_INDEX"]["@value"]["@value"][0]["@value"]
            else:
                argIndex = ""

            if "IS_EXTERNAL" in vertex["properties"]:
                isExternal = vertex["properties"]["IS_EXTERNAL"]["@value"]["@value"][0]
            else:
                isExternal = ""

            writer.writerow([key, type, code, name, location, locationEnd, functionId, argIndex, isExternal])

# JSON 파일을 처리하는 함수
def process_json_in_directory(directory):
    json_file_path = os.path.join(directory, "export.json")
    if os.path.exists(json_file_path):
        print(f"Processing {json_file_path}")
        data = read_json_file(json_file_path)
        edges = data["@value"]["edges"]
        vertices = data["@value"]["vertices"]
        
        edges_output_path = os.path.join(directory, "edges.csv")
        nodes_output_path = os.path.join(directory, "nodes.csv")
        
        Function_ids = write_edges_to_csv(edges, edges_output_path)
        write_vertices_to_csv(vertices, Function_ids, nodes_output_path)

# 디렉토리를 재귀적으로 탐색하는 함수
def process_directory_recursively(base_dir):
    for dir_name in os.listdir(base_dir):
        dir_path = os.path.join(base_dir, dir_name)

        # .c, .cpp 파일은 스킵
        if dir_name.endswith(('.c', '.cpp')):
            continue

        if os.path.isdir(dir_path):
            # 각 하위 디렉토리 탐색
            process_json_in_directory(dir_path)
            process_directory_recursively(dir_path)

# Main 함수
def main():
    base_dir = os.getcwd()
    process_directory_recursively(base_dir)

# 스크립트 실행
if __name__ == "__main__":
    main()

Processing c:\Users\csw24\Projects\code_slice_v2\parsing_dirs\parsing_CWE122_Heap_Based_Buffer_Overflow__c_CWE193_char_memmove\CWE122_Heap_Based_Buffer_Overflow__c_CWE193_char_memmove_01\export.json
Processing c:\Users\csw24\Projects\code_slice_v2\parsing_dirs\parsing_CWE122_Heap_Based_Buffer_Overflow__c_CWE193_char_memmove\CWE122_Heap_Based_Buffer_Overflow__c_CWE193_char_memmove_02\export.json
Processing c:\Users\csw24\Projects\code_slice_v2\parsing_dirs\parsing_CWE122_Heap_Based_Buffer_Overflow__c_CWE193_char_memmove\CWE122_Heap_Based_Buffer_Overflow__c_CWE193_char_memmove_03\export.json
Processing c:\Users\csw24\Projects\code_slice_v2\parsing_dirs\parsing_CWE122_Heap_Based_Buffer_Overflow__c_CWE193_char_memmove\CWE122_Heap_Based_Buffer_Overflow__c_CWE193_char_memmove_04\export.json
Processing c:\Users\csw24\Projects\code_slice_v2\parsing_dirs\parsing_CWE122_Heap_Based_Buffer_Overflow__c_CWE193_char_memmove\CWE122_Heap_Based_Buffer_Overflow__c_CWE193_char_memmove_05\export.json
Proce